In [22]:
#pip install msedge-selenium-tools selenium==3.141

In [23]:
import csv
from pymongo import MongoClient
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from msedge.selenium_tools import Edge, EdgeOptions

In [24]:

driver = webdriver.Chrome('C:/chromedriver.exe')



In [25]:
def get_url(search_text):
    """Generate a url from search text"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_text.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page={}'
        
    return url

In [26]:
#amazon
def extract_record(item):
    """Extract and return data from a single record"""
     
    # description and url
    atag = item.h2.a
    global description
    description= atag.text.strip()
    global url 
    url= 'https://www.amazon.com' + atag.get('href')
    try:
        # product price
        price_parent = item.find('span', 'a-price')
        global price 
        price = price_parent.find('span', 'a-offscreen').text

    except AttributeError:
        return
    
    try:
        # rating and review count
        global rating
        rating= item.i.text
        global review_count 
        review_count= item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result


In [27]:
def main(search_term):
    """Run main program routine"""
    
    records = []
    url = get_url(search_term)
    datum = {}
    for page in range(1,30):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                datum['price']=price
                datum['description']=description
                datum['rating']=rating
                datum['review_count']=review_count
                datum['url']=url
                #record.append(datum)
                records.append(record)
        #print(records)
    
    
    client = MongoClient("mongodb://127.0.0.1:27017") 
    db = client.mymongodb 

    #for company in records:
       
    #db.companies.insert_many(records)
      
    driver.close()
    
    # save data to csv file
    import csv
    with open('results=laptop.xlsx', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        header=['Description', 'Price', 'Rating', 'ReviewCount', 'Url']
        writer.writerow(header)
        for company in records:
            print(company)
            writer.writerow(company)


In [28]:
# run program
main('laptop')


('SGIN Laptop 15.6 Inch 12GB DDR4 512GB SSD, Windows 11 Laptops with Intel Celeron N4500, FHD 1920x1080, Dual Band WiFi, 2xUSB 3.0, Up to 2.8Ghz, Bluetooth 4.2, Supports 512GB TF Card Expansion, Gray', '$419.99', '', '', 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A04805432ZB9210OYNL1I&url=%2FSGIN-1920x1080-Bluetooth-Supports-Expansion%2Fdp%2FB09XQZMS64%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptop%26qid%3D1654334600%26sr%3D8-1-spons%26psc%3D1&qualifier=1654334600&id=1404998964440953&widgetName=sp_atf')
('Acer Nitro 5 AN515-57-79TD Gaming Laptop | Intel Core i7-11800H | NVIDIA GeForce RTX 3050 Ti Laptop GPU | 15.6" FHD 144Hz IPS Display | 8GB DDR4 | 512GB NVMe SSD | Killer Wi-Fi 6 | Backlit Keyboard', '$998.82', '', '', 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05951653GBLL6ZRENUL4&url=%2FAcer-AN515-57-79TD-i7-11800H-GeForce-Keyboard%2Fdp%2FB09R65RN43%2Fref%3Dsr_1_2_sspa%3Fkeyword